# Homework 2

For this homework, you'll re-write the `Parser` class.  Instead of using keyword based parsing, as was done in old-school text adventure games, we'll see how we can use large language models (LLMs) like GPT.  This will allow us to do two important things:

1. We can use the LLM to generate more evocative descriptions.
2. We can do more flexible matching of user commands.

## Setting up the OpenAI API

You'll need to run `python -m pip install openai` in your terminal, and specify your OpenAI API Key (which you can find [here](https://platform.openai.com/api-keys)) on the terminal with the command `export OPENAI_API_KEY=sk-##########`. If you're using VS Code, you can launch it from your terminal with the `code` command and prefix it with this export statement on the same line.  For example:

```
cd your/homework/dir
source venv/bin/activate
OPENAI_API_KEY=sk-##########  code .
```


In [1]:

from openai import OpenAI

import os
from getpass import getpass

if 'OPENAI_API_KEY' not in os.environ:
    print("You didn't set your OPENAI_API_KEY on the command line.")
    os.environ['OPENAI_API_KEY'] = getpass("Please enter your OpenAI API Key: ")


You didn't set your OPENAI_API_KEY on the command line.


## Your first call to the OpenAI API



In [2]:
from openai import OpenAI
client = OpenAI()

# response = client.chat.completions.create(
#   model="gpt-4",
#   messages=[
#     {
#       "role": "system",
#       "content": "You are the narrator for a text adventure game. You create short, evocative descriptions of the scenes in the game. Use a style like Jane Austen."
#     },
#     {
#       "role": "user",
#       "content": "look"
#     },
#     {
#       "role": "assistant",
#       "content": "You are standing in a small cottage.\nExits:\nOut to Garden Path\n\nYou see:\n * a fishing pole\n * a poisonous potion\n"
#     }
#   ],
#   temperature=1,
#   max_tokens=256,
#   top_p=1,
#   frequency_penalty=0,
#   presence_penalty=0
# )

In [3]:
import textwrap
def wrap_text(text: str, width: int = 80) -> str:
    """
    Keeps text output narrow enough to easily be read
    """
    lines = text.split("\n")
    wrapped_lines = [textwrap.fill(line, width) for line in lines]
    return "\n".join(wrapped_lines)

In [4]:
# print(response.choices[0].message.content)

In [5]:
# print(wrap_text(response.choices[0].message.content))

## Environment

Next, let's import our text adventure games. 

In [4]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

### Part 1: Using GPT to Write Descriptions

The text of our adventures in HW1 was a bit bland.  We will can make it more evocative with GPT by subclassing the game's default `Parser` class and having our new `GptParser` subclass override the `ok` and `fail` methods that describe things to the player.

In [6]:
import json
import tiktoken

class GptParser(parsing.Parser):
    def __init__(self, game, echo_commands=True, verbose=False):
        super().__init__(game, echo_commands=echo_commands)
        self.verbose=verbose
        self.client = OpenAI()
        self.gpt_model = "gpt-4"
        # self.gpt_model = "gpt-3.5-turbo"
        self.max_output_tokens = 256 # You get to pick this
        self.max_tokens = 8192-self.max_output_tokens # GPT-4's max total tokens
        self.tokenizer = tiktoken.get_encoding("cl100k_base")


    def gpt_describe(self, system_instructions, command_history):
        """
        Generate a description with GPT.  This takes two arguments:
        * The system instructions, which is the prompt that describes 
          how you'd like GPT to behave.
        * The command history - this is a list of previous user input 
          commands and game descriptions. It's given as context to GPT.
        The Parser class manages the command_history via the 
        `add_command_to_history` and `add_description_to_history` functions
        which use the ChatGPT format with commands being assigned role: user,
        and descriptions being assigned role: assistant.
        """
        try:
            messages = [{
                "role": "system",
                "content": system_instructions
            }]
            context = self.limit_context_length(command_history, self.max_tokens)
            messages.extend(context)
            if self.verbose:
                print(json.dumps(messages, indent=2))
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                temperature=1,
                max_tokens=self.max_output_tokens,
                top_p=0,
                frequency_penalty=0,
                presence_penalty=0
            )
            content = response.choices[0].message.content
            return content
        except Exception as e:
            return f"Something went wrong with GPT: {e}"
        
    def limit_context_length(self, command_history, max_tokens, max_turns=1000):
        if len(command_history) >= max_turns:
            return command_history[-max_turns:]

        num_tokens = 0
        for i in range(len(command_history) - 1, -1, -1):
            num_tokens += len(self.tokenizer.encode(command_history[i]['content']))
            if num_tokens >= max_tokens:
                return command_history[-i:]
        return command_history

    
    def ok(self, description):
        """
        In this homework, we'll replace this with a call to the OpenAI API
        in order to create more evocative descriptions.  For this part, 
        all you need to do is create your own system instructions.   
        
        You should: 
        1. Design system_instructions to pass to GPT to narrate the game based
           on the user's command, and the game's default response. 
        2. Query GPT using your gpt_describe and self.command_history
        3. Print GPT's output (optionally using self.wrap_text)
        4. Add GPT's output to the command history using add_description_to_history.
        """
        self.add_description_to_history(description)
        system_description = """You are the narrator for a text adventure game. 
            You create violent, evocative descriptions of the scenes in the game. 
            The description of the scene should speak like John Wick.
            Use what's in your inventory, your knowledge of the world, 
            and your luck. Here is the user's input. """
        system_description = system_description.replace("\n", "")
        response = self.gpt_describe(system_instructions=system_description, command_history=self.command_history)
        # response = "This action succeeds."
        response = self.wrap_text(response)
        self.add_description_to_history(response)
        print(response)        

    def fail(self, description):
        """
        In this homework, we'll replace this with a call to the OpenAI API
        in order to create more useful error messages descriptions.  You should
        do the same steps as for the ok() command, but optionally customize 
        the system_instructions to tell the user that their command failed.
        """
        self.add_description_to_history(description)
        system_description = """You are the narrator for a text adventure game. 
            You create violent, evocative descriptions of the scenes in the game. 
            The description of the scene should speak like John Wick.
            Use your guns, your knowledge of the world, 
            and your luck. Here the user put in invalid input, 
            so you need to tell them that their command failed, 
            but in a way that is consistent with the game's style."""
        # system_description = system_description.replace("\n", "")
        response = self.gpt_describe(system_instructions=system_description, command_history=self.command_history)
        response = "This action fails."
        response = self.wrap_text(response) 
        self.add_description_to_history(response)
        response = self.wrap_text(response)
        print(response)

### Describing Action Castle

Let's use our new system prompt to have GPT generate descriptions of Action Castle!  We'll import my implementation of the Action Castle game, and then run the game with your GptParser.

In [7]:
import hw1_solution.action_castle as action_castle

game = action_castle.build_game()
game.give_hints = True
parser = GptParser(game, verbose=False)
game.set_parser(parser)

In [9]:
# game.game_loop()

You should do a play through of Action Castle, and save a trascript of your play to submit along with your homework solution.  I recommend trying several commands that don't work.  Also, be on the look out for "hallucinations" by GPT, which in this context are elaborations that it adds that aren't part of the underlying game.  Try to interact with hallucinated objects and see what happens.  Is the underlying game state maintained?

In [8]:
import json
import datetime

def save_game_transcript(game, filename_prefix="game_transcript"):
    current_datetime = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    filename = f"{filename_prefix}-{current_datetime}.json"
    with open(filename, 'w') as file:
        # Pretty-print and save the command history
        json.dump(game.parser.command_history, file, indent=4)
        print(f"Game transcript saved successfully to {filename}")
 

save_game_transcript(game, "game_transcript_with_GPT_descriptions")

Game transcript saved successfully to game_transcript_with_GPT_descriptions-2024-02-20_22-12.json


## Part 2: Intent Determination  

The keyword-based parser for our HW1 text adventure game could only match very specific commands.  We could win the fishing mini game with this sequence of commands: 
1. take pole
2. go out
3. south
4. catch fish with pole
5. eat fish

However, if the user tried some other very similar commands, the simple keyword based parser wouldn't understand the meaning of these commands.

1. pick up the fishing pole
2. head outside
3. go down to the fishing pond
4. go fishing with the pole
5. I'm starving so I swallow the fish whole 

Try these commands using your GptParser – it won't work properly without special handling.  In fact, you might get out of sync with the game state. 

In this part of the homework, we'll extend your GptParser to more flexibly match user commands.  For this part of the homework, we'll implement a new version of the `determine_intent`.

There are several possible ways that you could flexibly match a user's input. 

* You could try using the [emeddings API](https://platform.openai.com/docs/guides/embeddings) 
* You could ask GPT-4 to pick the closest matching output from a list that you provide. 
* You could fine-tune GPT-3 to perform the task using [the fine-tuning API](https://platform.openai.com/docs/guides/fine-tuning). 

For my solution,  picked the second one. To ensure that I returned 
exactly the ACTION_NAME and not some longer text, I had to carefully craft 
my instructions to GPT, and eventually I resorted to using a regular 
expression over GPT's output).   

In [11]:
class GptParser2(GptParser):
    def __init__(self, game, echo_commands=True, verbose=False):
        super().__init__(game, echo_commands, verbose)
        # Command descriptions is a dictionary that maps
        # action descriptions and aliases onto action names 
        self.command_descriptions = {}
        for _, action in self.actions.items():
            description = action.ACTION_DESCRIPTION
            if action.ACTION_ALIASES:
                description += " (can also be invoked with '{aliases}')".format(
                    aliases = "', '".join(action.ACTION_ALIASES)
                )
            action_name = action.ACTION_NAME
            if action_name:
                self.command_descriptions[description] = action_name


    def determine_intent(self, command):
        """
        Given a user's command, this method returns the ACTION_NAME of
        the intended action.  
        """

        command = command.lower()
        if "," in command:
            return "sequence"

        action_list = ""

        for _, action in self.actions.items():
            a = action.ACTION_NAME + ", "
            action_list += a

        
        # action_list += "unlock door, read runes, propose, wear crown, sit on throne, "

        instruction = ("The following is the player's action: \"" + command 
                       + "\". The following is a list of possible actions, [" + action_list +
                       "] and you should find an action from the list that matches best with the player's action, "
                       "then output it, using the exact same wording, and nothing extra. "
                       "Pay attention to both nouns and verbs in the action." 
                       )
        messages = [
            {
            "role": "system",
            "content": """You are a helpful paraphraser. You take in a user's command, 
                and return the action name of the intended action."""
            },
            {
            "role": "user",
            "content": instruction
            }
        ]
        response = self.client.chat.completions.create(
            model=self.gpt_model,
            messages=messages,
            temperature=1,
            max_tokens=self.max_output_tokens,
            top_p=0,
            frequency_penalty=0,
            presence_penalty=0
        )
        
        content = response.choices[0].message.content
        # print(content)
        return content
        # TODO - implement this function 
    




### Test your determine_intent method.  

Feel free to incrase the number of test cases.  Describe your testing in the README that you submit for your homewokr. 

In [13]:
game = action_castle.build_game()
parser = GptParser2(game, verbose=False)
game.set_parser(parser)

test_cases = {
    "pick up the fishing pole": "get",
    "head outside": "go",
    "travel to the fishing pond": "go",
    "go fishing with the pole": "catch fish",
    "I'm starving so I swallow the fish whole": "eat",
    "pick rose": "pick rose"
}

correct = 0
total = 0
for command, action_name in test_cases.items():
    print(">", command)
    predicted_action = parser.determine_intent(command)
    total += 1
    if predicted_action == action_name:
        print(f"Correct: determine_intent predicted {predicted_action}")
        correct += 1
    else:
        print(f"Incorrect: determine_intent predicted {predicted_action} but it should have returned {action_name}")
print(f"{correct} out of {total} were correctly predcited.")

> pick up the fishing pole


APIConnectionError: Connection error.

## Part 3: Argument Matching

After we've determined which of the Actions the user wants to perform, we need to flexibly match its arguments. If I user says "get the fishing rod", we want to execute the game command "get pole" even though the user didn't use the word "pole".

In this part, you should use GPT to implement the parser commands  `get_character`, `match_item` and `get_direction`.


In [15]:

from text_adventure_games.things.characters import Character
from text_adventure_games.things.items import Item
from text_adventure_games.things.locations import Location


class GptParser3(GptParser2):
    def __init__(self, game, echo_commands=True, verbose=False):
        super().__init__(game, echo_commands, verbose)

    def get_character(
        self, command: str, hint: str = None, split_words=None, position=None
    ) -> Character:
        """
        This method tries to match a character's name in the command.
        If no names are matched, it defaults to `game.player`. 
        Args:
            hint: A hint about the role of character we're looking for 
                  (e.g. "giver" or "recipent")
            split_words: not needed for our GptParser
            position: not needed for our GptParser
        """
        name_list = ""
        
        for name in self.game.characters.keys():
            n = name + ", "
            name_list += n

        if hint:
            instruction = f"This is the player's action: {command} \
                            The following is a list of possible characters, {name_list} \
                            and the narrator should find the {hint} of the action from the list, \
                            then output this {hint}, using only one word. If there is no {hint} of the action, \
                            then output the word \"player\". " 
        else:
            instruction = f"This is the player's action: {command} \
                            The following is a list of possible characters, {name_list} \
                            and the narrator should find a character from the list that matches with the player's action, \
                            then output this character, only one word. If there is no target character, \
                            then output the word \"player\". " 

        messages = [
            {
            "role": "system",
            "content": """You are a helpful paraphraser. You take in a user's command, 
                and return target character of the intended action."""
            },
            {
            "role": "user",
            "content": instruction
            }
        ]
        response = self.client.chat.completions.create(
            model=self.gpt_model,
            messages=messages,
            temperature=1,
            max_tokens=self.max_output_tokens,
            top_p=0,
            frequency_penalty=0,
            presence_penalty=0
        )

        predicted_character = response.choices[0].message.content

        predicted_character = predicted_character.replace('"', '').replace("'", "").lower()
            
        print(predicted_character)

        if predicted_character == "player":
            return self.game.player
        elif predicted_character in self.game.characters:
            return self.game.characters[predicted_character]
        else:
            return self.game.player
        # TODO - implement this method


    def match_item(
        self, command: str, item_dict: dict[str, Item], hint: str = None
    ) -> Item:

        item_list = ""

        for item_name in item_dict:
            item_name += ", "
            item_list += item_name
        
        print(item_list)

        instruction = f"""This is the player's action: {command}. \
            The following is a list of items, {item_list}, you should find the item mentioned by the user's action from the list. \
            Output the item's name, using exactly the wording of the item in the list, and only one word. \
            If there is no item mentioned or not matched with the list, output \"none\". \
            Remember, small item like fish is a valid item, but large things like a troll is not an item."""
        
        messages = [
            {
            "role": "system",
            "content": """You are a helpful paraphraser. You take in a user's command, 
                and return the target item that is mentioned in the action."""
            },
            {
            "role": "user",
            "content": instruction
            }
        ]
        response = self.client.chat.completions.create(
            model=self.gpt_model,
            messages=messages,
            temperature=1,
            max_tokens=self.max_output_tokens,
            top_p=0,
            frequency_penalty=0,
            presence_penalty=0
        )

        predicted_item = response.choices[0].message.content

        predicted_item = predicted_item.replace('"', '').replace("'", "").lower()
        # print("predicted item: ", predicted_item)

        if predicted_item != "none" and predicted_item in item_dict:
            return item_dict[predicted_item]
        else:
            return None

        #return None # TODO - implement this method



    def get_direction(self, command: str, location: Location = None) -> str:
        """
        Return the direction from `location.connections` which the player
        wants to travel to.
        """
        instruction = ("The following is the player's action: \"" + command + 
            "\". A possible list of directions: north, west, south, east, up, down, out, in. "
            "Decide if the player intends to go any direction. "
            "If yes, output the direction, only one word from the list, nothing else; if no, output an empty string."
            )
        messages = [
            {
            "role": "system",
            "content": """You are a helpful paraphraser. You take in a user's command, 
                and return the target direction of the intended action."""
            },
            {
            "role": "user",
            "content": instruction
            }
        ]
        response = self.client.chat.completions.create(
            model=self.gpt_model,
            messages=messages,
            temperature=1,
            max_tokens=self.max_output_tokens,
            top_p=0,
            frequency_penalty=0,
            presence_penalty=0
        )

        predicted_direction = response.choices[0].message.content

        predicted_direction = predicted_direction.replace('"', '').replace("'", "").lower()

        if predicted_direction:
            # print(predicted_direction)
            return predicted_direction
        else:
            return None

        # return None # TODO - implement this method



Next, do another playthrough of the game and try to use only commands that do not correspond to the ones we had in HW1.  Save your trascript and upload it.

In [28]:
import hw1_solution.action_castle as action_castle

game = action_castle.build_game()
game.give_hints = True
parser = GptParser3(game, verbose=False)
game.set_parser(parser)
game.game_loop()

> look
sequence, attack, catch fish, describe, drink, drop, eat, examine, get, give, go, inventory, light, pick rose, quit, smell rose, unlock door, read runes, propose, wear crown, sit on throne, 
examine
player
pole, lamp, 
predicted item:  none
This action succeeds.


In [17]:

save_game_transcript(game, "game_transcript_with_flexible_commands")

Game transcript saved successfully to game_transcript_with_flexible_commands-2024-02-20_22-05.json


## Optional Extension: Comparing Approaches to Intent Determination 

If you're working in a group larger than a pair, then you should pick one optional extension for each group member above 2.  One extension that you could do is to compare several approaches to intent determination, and quanify how good each approach is on an extended test set that you put together.



## Optional Extension: Image creation 

In addition to creating evocative text with the OpenAI API, you can also use it to create images with DALLE-3.  Below are some examples.  If you do this extension, you should pre-generate images for each of the locations in your game, and save them.  Then you can display them when a character moves into a location.  You can generate several and choose your favorites.  In your writeup, describe what prompts you used, and what you did to try to keep a consistent image style throughout the game.

In [ ]:


img_prompts = [
"""
Style: Text Adventure Game like Zork.

You are standing in a small cottage.
Exits:
Out to Garden Path

You see:
 * a fishing pole
 * a poisonous potion

""",

"""
Style: Jane Austen

As you gaze around, you find yourself ensconced within a humble yet enchanting little cottage, stowed away like a precious secret in the heart of the country. The air is redolent with the comforting aroma of wood smoke and freshly baked bread, warming the cockles of one's heart.

The cottage breathes and lives through its evidence of mundane life: a neglected fishing pole that must have had stories old and new twined around its line, resting against the wall near the window, almost in anticipation of the next adventure. On a sturdy oak table below the window, stands a mysteriously dark vial, the poisonous potion it houses shimmering under the faint candlelight.

At your disposal, my dear adventurer, lies the exit – a door, simple yet inviting, promising the rustic charm of a garden path beyond its threshold. Whatever path you choose, remember, curiosity tempered with caution is the true calling of an adventurer. Through the looking glass, my dear, through the looking glass.
""",


"""
Style: Edgar Allen Poe

Encircled by the foreboding veil of twilight, you find yourself within the confines of a quaint, yet somewhat desolate cottage. Tattered curtains tremble as a chilly breeze invades the space, and the lowly lit fire casts a lurid flicker across the room, spawning grotesque shadows on the bare stone walls. 

Options lie before you:
Paths lead you back towards the sinuous charm of the whispering garden from whence you came.

Upon your survey, objects of intrigue claw out from the gloom:
* A fishing pole, a skeleton of erstwhile merriment, it lay unused and embraced by layers of dust. 
* A pulsating vial is also there, a phantom-like glow emanates from it, the liquid within as deadly as the most lethal of venoms, tinging the air with an atmosphere of potent danger and allure.
""",



"""
Style: William Shakespeare 

Thou dost find thyself within the humble confines of a modest cottage, seeming born from the earth herself. 'Tis a space small in size, yet grand in its cosiness and peace, imbued with the warmth of bygone days.

Exit there are but one:
Yonder lies the verdant garden path, serenading with the soft whispers of the zephyr.

Present sights before thee are:
* A fishing pole, lean and patient as if anticipating the secret dance of the river's denizens.
* A draught of venomous potion, dancing with dread darkness and fey mystery.
""",

"""
Style: Donald Trump

You're in this beautiful, yuge cottage folks, it's just perfect. We've got the most beautiful cottages, I'll tell you that.

Look, here's how you get around: You've got this charming little garden path. And what a garden path folks, it’s like nothing you've ever seen before, just splendid.

Now look here, you see that? You've got two things in here:

1. A fishing pole. And it's not just any fishing pole. It's a totally first-class, top-notch, best you've ever seen fishing pole. 

2. Watch out though! There’s this totally toxic, a disastrous potion. Trust me it's no good, folks. It's just poison in a pretty bottle. I wouldn't touch it if I were you. It’s a disaster!
""",

"""
Style: Young Adult Dystopian Fiction

You find yourself in a ramshackle one-room cottage, abandoned long ago by anyone who once entertained life's ordinary pleasures. It is dusty and forlorn yet somehow undeniably cozy. The late afternoon sunlight seeps weakly through the moth-eaten curtains, painting patterns on the worn wooden floorboards that hint of forgotten laughter and secrets whispered in the night. 

To the north, the door, barely hanging on its hinges, leads to what was once a charming garden path, now reclaimed by the unyielding grasp of nature; brambles and twisted vines run rampant, marking the passage of time that mankind forgot.

Around you, a few relics hold out stubbornly against the annihilation of civilization. A forsaken fishing pole rests against the shadowy corner, whispering tales of tranquil days, while a vial of radiant, emerald-green liquid - a poisonous potion, no doubt concocted in the crucible of man's unquenchable thirst for dominance - is more ominous, sitting quietly on the rustic table like a ticking time bomb.
""",


"""
Style: Pulp Fiction and Samuel L. Jackson

Look around and what do you see, motherfucker? We're in this tiny-ass cottage, feels like you could touch each wall if you stretched your arms wide. It's got that cozy vibe going on, but don't let it fool you, nothing's ever what it seems.

You've got two ways out. One is the way you came in...a garden path. Tranquil as shit, green, just like you'd imagine. 

Now here's the fun part. A goddamn fishing pole is sitting there, all cool, waiting to be picked up. You know when you see a fishing pole that shit's about to get interesting. 

And what's this? A damn poisonous potion. Don't ask me why that's there, I'm just the Samuel L. Jackson flavoured voice in your head. You decide what to do with it. But whatever it is, you'd better reckon it fast, motherfucker.
""",

]



In [ ]:
from openai import OpenAI

client = OpenAI()

image_urls = []

for img_prompt in img_prompts:

    response = client.images.generate(
      model="dall-e-3",
      prompt=img_prompt,
      size="1792x1024",
      quality="standard",
      n=1,
    )
    image_url = response.data[0].url
    print(image_url)
    image_urls.append(image_url)

## Optional Extension: Redefine ActionSequence

When I was playing my game, I noticed that action sequences weren't being handled correctly.  For instance, in the cottage I typed "head to the garden and then go to the fishing pond".  Because the ActionSequence currently only seperates commands based on commas, the game didn't actually execute the second part.  But GPT hallucinated that I was at the fishing pond, even though the game thought I was still in the garden.  GPT got really out of sync with the game.

```json
        "role": "user",
        "content": "head to the garden and then go to the fishing pond"
    },
    {
        "role": "assistant",
        "content": "You are standing on a lush garden path. There is a cottage here.\nExits:\nIn to Cottage\nSouth to Fishing Pond\nNorth to Winding Path\n\nYou see:\n * a rosebush\n\tpick rose\n\n"
    },
    {
        "role": "assistant",
        "content": "You step out of the cottage and onto a garden path. The air is fresh and filled with the scent of blooming flowers. You follow the path, your footsteps crunching on the gravel underfoot. The path leads you south, away from the cottage and towards a serene fishing pond. The pond is surrounded by tall, whispering reeds and the water is as clear as glass. You can see fish darting beneath the surface. It's a perfect spot for fishing."
    },
    {
        "role": "user",
        "content": "look"
    },
    {
        "role": "assistant",
        "content": "You are standing on a lush garden path. There is a cottage here.\nExits:\nIn to Cottage\nSouth to Fishing Pond\nNorth to Winding Path\n\nYou see:\n * a rosebush\n\tpick rose\n\n"
    },
    {
        "role": "assistant",
        "content": "You are standing by the serene fishing pond. The water is calm, reflecting the clear blue sky above. You can see fish swimming lazily beneath the surface, their scales glinting in the sunlight. Tall reeds sway gently in the breeze at the water's edge, and a few ducks paddle peacefully in the distance. The air is filled with the sound of chirping birds and the soft rustle of leaves. It's a perfect spot for a quiet afternoon of fishing."
    },
    },
    {
        "role": "user",
        "content": "go back to the garden"
    },
    {
        "role": "assistant",
        "content": "Garden path does not have an exit 'None'"
    },
    {
        "role": "assistant",
        "content": "I'm sorry, but you can't go back to the garden from here. You need to use the command \"go north\" to return to the garden path."
    },
    {
        "role": "user",
        "content": "n"
    },
    {
        "role": "assistant",
        "content": "You are walking along a winding path. There is a tall tree here.\nExits:\nSouth to Garden Path\nUp to Top of the Tall Tree\nEast to Drawbridge\n\n\n\n"
    },
    {
        "role": "assistant",
        "content": "You leave the pond behind and head north, back along the garden path. The path winds through a riot of colorful flowers and lush greenery, the air filled with the sweet scent of blooming roses and the soft hum of bees. You find yourself back at the garden, the cottage visible in the distance."
    },
    {
        "role": "user",
        "content": "in"
    },
    {
        "role": "assistant",
        "content": "Winding path does not have an exit 'in'"
    },
    {
        "role": "assistant",
        "content": "I'm sorry, but you can't go 'in' from here. You are currently on a winding path. The available directions you can go are south back to the garden path, up to the top of the tall tree, or east to the drawbridge."
    },
    {
        "role": "user",
        "content": "quit"
    }
```

One way of fixing this would be to GPT to seperate the commands (e.g. by re-writing the command to include commas) before creating an ActionSequence.  So a command like "I want to head to the garden and then go to the fishing pond. After I catch a fish with my pole then go back to the garden again." would get rewritten as "I want to head to the garden, go to the fishing pond, catch a fish with my pole, go back to the garden again".